In [1]:
import json
import numpy as np
from tqdm import tqdm
from pprint import pprint
from sklearn.cluster import KMeans

from label_processor import CStdLib

In [2]:
with open('result_logits.json', encoding='utf-8') as f:
    logits_json = json.load(f)

In [3]:
embed = CStdLib(single=False)

In [4]:
pbar = tqdm(logits_json.items())
i = 0
for key, prod_dict in pbar:
    optional_tags = prod_dict['optional_tags']
    num_class = len(optional_tags)
    imgs_tags = prod_dict['imgs_tags']
    imgs_logits = []
    for i in range(len(imgs_tags)):
        imgs_logits.append(np.fromstring(imgs_tags[i][key + '_{}.jpg'.format(i)], dtype=np.float32, sep=' '))
    imgs_logits = np.array(imgs_logits)
    
    # 聚类
    cluster = KMeans(n_clusters=num_class, random_state=42).fit(imgs_logits)
    labels = cluster.labels_
    
    # 计算匹配度
    match_score = np.zeros((num_class, num_class))
    # match_score[i, j] 表示 kmeans 分类结果为 i 和 optional_tags[j] 的匹配度
    embeded_tags = [embed(tag) for tag in optional_tags]
    not_matched = False
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        logits = imgs_logits[id_i]
        for j in range(num_class):
            logits_ids = embeded_tags[j]
            if len(logits_ids) > 0:
                logits_j = logits[:, logits_ids]
                match_score[i, j] = np.mean(logits_j)
            else:
                not_matched = True
                match_score[i, j] = 1
    
    # 对 scores 进行归一化，使得每一列的 ord 范数为 1；然后在每一行中找到最大值，即为该行商品对应的标签。
    # 按列归一化，按行找最大值，这样就做到了双向选择。
    # 使用较高阶的范数能增大未匹配标签对应的概率，有利于提高 acc/em。
    match_score = (np.abs(match_score) +  match_score) / 2. + 1e-7 # 只保留正数，加 1e-7 避免 warning
    match_score_norm = match_score / np.linalg.norm(match_score, axis=0, ord=3)
    # print(key, '\n', match_score_norm)
    label_id = np.argmax(match_score_norm, axis=1)
    label_score = np.max(match_score_norm, axis=1)
    # print(label_id)
    
    # 强制使每一类都被选中
    selected_label_id = []
    for i in range(num_class):
        if label_id[i] == -1:
            continue
        selected_label_id.append(label_id[i])
        for j in range(i + 1, num_class):
            if label_id[i] == label_id[j]:
                if label_score[i] > label_score[j]:
                    label_id[j] = -1
                else:
                    label_id[i] = -1
    selected_label_id = set(selected_label_id)
    unselected = list(set(range(num_class)) - selected_label_id)
    i = 0
    for j in range(num_class):
        if label_id[j] == -1:
            label_id[j] = unselected[i]
            i += 1
    
    # 终于可以写入 json 了
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        for j in id_i:
            logits_json[key]['imgs_tags'][j][key + '_{}.jpg'.format(j)] = optional_tags[label_id[i]]
    if not_matched:
        print(key, '\n', match_score, '\n', match_score_norm)
        pprint(logits_json[key])

  2%|▏         | 96/5331 [00:00<00:42, 124.55it/s]

623030476282 
 [[6.47678147e+00 1.00000010e+00]
 [1.00000000e-07 1.00000010e+00]] 
 [[1.00000000e+00 7.93700526e-01]
 [1.54397675e-08 7.93700526e-01]]
{'imgs_tags': [{'623030476282_0.jpg': '9954黑色西装外套'},
               {'623030476282_1.jpg': '9954黑色西装外套'},
               {'623030476282_2.jpg': '9953条纹衬衣'},
               {'623030476282_3.jpg': '9954黑色西装外套'},
               {'623030476282_4.jpg': '9953条纹衬衣'},
               {'623030476282_5.jpg': '9954黑色西装外套'},
               {'623030476282_6.jpg': '9954黑色西装外套'}],
 'optional_tags': ['9954黑色西装外套', '9953条纹衬衣']}


  5%|▌         | 281/5331 [00:02<01:02, 80.72it/s] 

624952669722 
 [[1.00000000e-07 1.00000010e+00]
 [4.34332113e-01 1.00000010e+00]] 
 [[2.30238559e-07 7.93700526e-01]
 [1.00000000e+00 7.93700526e-01]]
{'imgs_tags': [{'624952669722_0.jpg': '珊瑚色组合'},
               {'624952669722_1.jpg': '珊瑚色组合'},
               {'624952669722_2.jpg': '橄榄色组合'},
               {'624952669722_3.jpg': '橄榄色组合'},
               {'624952669722_4.jpg': '橄榄色组合'}],
 'optional_tags': ['珊瑚色组合', '橄榄色组合']}


  6%|▋         | 338/5331 [00:03<01:08, 72.87it/s] 

...

 85%|████████▌ | 4533/5331 [00:42<00:06, 119.98it/s]

577541680180 
 [[9.96545802e+00 1.00000010e+00 2.70467554e+00 1.00000000e-07]
 [1.00000000e-07 1.00000010e+00 1.00000000e-07 8.64553462e+00]
 [1.08791361e+00 1.00000010e+00 7.65686665e+00 1.00000000e-07]
 [1.00000000e-07 1.00000010e+00 1.00000000e-07 3.29344402e+00]] 
 [[9.99566695e-01 6.29960525e-01 3.48193124e-01 1.13610582e-08]
 [1.00303136e-08 6.29960525e-01 1.28737484e-08 9.82224217e-01]
 [1.09121148e-01 6.29960525e-01 9.85725750e-01 1.13610582e-08]
 [1.00303136e-08 6.29960525e-01 1.28737484e-08 3.74170091e-01]]
{'imgs_tags': [{'577541680180_0.jpg': '红色'},
               {'577541680180_1.jpg': '红色'},
               {'577541680180_2.jpg': '蓝色'},
               {'577541680180_3.jpg': '蓝色'},
               {'577541680180_4.jpg': '橙色'},
               {'577541680180_5.jpg': '深色'},
               {'577541680180_6.jpg': '橙色'},
               {'577541680180_7.jpg': '蓝色'},
               {'577541680180_8.jpg': '蓝色'}],
 'optional_tags': ['橙色', '深色', '红色', '蓝色']}


 90%|████████▉ | 4780/5331 [00:44<00:04, 120.03it/s]

612126042525 
 [[1.0000001  6.93167267]
 [1.0000001  3.33070503]] 
 [[0.79370053 0.96553849]
 [0.79370053 0.4639463 ]]
{'imgs_tags': [{'612126042525_0.jpg': '957黑色'},
               {'612126042525_1.jpg': '957黑色'},
               {'612126042525_2.jpg': '957黑色'},
               {'612126042525_3.jpg': '6802格子'},
               {'612126042525_4.jpg': '957黑色'},
               {'612126042525_5.jpg': '957黑色'},
               {'612126042525_6.jpg': '957黑色'}],
 'optional_tags': ['6802格子', '957黑色']}


100%|██████████| 5331/5331 [00:49<00:00, 107.62it/s]


In [5]:
with open('./result_labels.json', 'w') as f:
    json.dump(logits_json, f, indent=4, ensure_ascii=False)